In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random
!pip install -q efficientnet
import efficientnet.keras as effnet

### Declaring constants

In [ ]:
AUTOTUNE=tf.data.experimental.AUTOTUNE
IMG_SIZE=299
BATCH_SIZE=16
TFRECS_FORMAT={'image': tf.io.FixedLenFeature([], tf.string),
                      'image_name': tf.io.FixedLenFeature([], tf.string),
                      'target': tf.io.FixedLenFeature([], tf.int64)}

### Reading TFRecords and other utility functions

In [ ]:
@tf.function
def readTFRecs(dir_name):
    TFRecFiles=tf.constant(tf.io.gfile.listdir(dir_name))
    TFRecFiles=tf.map_fn(lambda name:dir_name+'/'+name,TFRecFiles)
    TFRecDataset=tf.data.TFRecordDataset(TFRecFiles)#.batch(self.BATCH_SIZE).prefetch(1)
    Dataset = TFRecDataset.map(lambda example:tf.io.parse_example(example,TFRECS_FORMAT))
    return Dataset
    
@tf.function
def decode_image(entry):
    return tf.cast(tf.image.decode_jpeg(entry['image'],channels=3),tf.float32),tf.cast(tf.one_hot(entry['target'],5),tf.float32) #[batch_size,h,w,3]    
    
def makeDataset(filename):
    TFRecDataset=readTFRecs(filename)
    Dataset = TFRecDataset.map(lambda entry: decode_image(entry))
    #Dataset = Dataset.shuffle(1000)
    #Dataset=Dataset.zip(TFRecDataset.map(lambda entry:entry['target']))
    Dataset = Dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    
    return Dataset


    


In [ ]:
dataset = makeDataset('../input/cassava-tfrecords-512x512')
valDataset = dataset.take(50)
trainDataset = dataset.skip(50)
del dataset


### Augmentations using @tf.function

We will use keras layer for including gaussian noise


In [ ]:


#@tf.function
#@tf.function
def get_augs():
    aug_list = np.random.randint(2, size=6)
    aug_list = list(map(lambda x:x>0,aug_list))
    return aug_list
    
#@tf.function
def resize_batch(images):
    return tf.image.resize(images,[IMG_SIZE,IMG_SIZE])

    
    
#@tf.function
def augment_batch_randomly(imgs,labels):
    '''
    Augmentaions to be used: (use stateless versions of these)
    
    Random hue (0.2)
    Random brightness (0.3)
    Random saturation (0.7,1.3)
    Random contrast  (0.8,1.2)
    ''' 
    augment_list=get_augs()
    images = resize_batch(imgs)
     #(32,512,512,3)
    
    if augment_list[0]:
        images = tf.image.random_saturation(images,0.7,1.3)
    if augment_list[1]:
        images = tf.image.random_contrast(images,0.8,1.2)
    if augment_list[2]:
        images = tf.image.random_brightness(images,0.3)
    if augment_list[3]:
        images = tf.image.random_hue(images,0.2)
    if augment_list[4]:
        images = tf.image.random_flip_left_right(images)
    if augment_list[5]:
        images = tf.image.random_flip_up_down(images)
    
    images = tf.math.divide(images,255)
        
    return images,labels
    

### Map dataset with augmentations

In [ ]:
trainDataset = trainDataset.map(augment_batch_randomly)
valDataset = valDataset.map(augment_batch_randomly)

### Make model

In [ ]:
filepath = "effnet4_aug-{epoch:02d}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')


In [ ]:
model = tf.keras.Sequential()
efn_model = effnet.EfficientNetB4(weights='noisy-student')
base_model = tf.keras.models.Model(inputs=efn_model.input, outputs=efn_model.layers[-3].output)
base_model.trainabe = True
model.add(tf.keras.Input((299,299,3)))
model.add(base_model)
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(10,activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(5,activation='softmax'))

model.compile(optimizer='adam',
             loss=tf.keras.losses.CategoricalCrossentropy(),
             metrics=[tf.keras.metrics.CategoricalAccuracy()])

model.summary()

In [ ]:
history = model.fit(trainDataset,epochs=15,validation_data=valDataset,callbacks=[checkpoint],verbose=1)